In [2]:
# use the raw data :')
import mne
import numpy as np
import pandas as pd
import scipy
import re
import random

# some hyperparameter type things :)
sr = 200
t = 200
ch = 19
affix = '_raw'

In [3]:
# load from .pkl files ("raw")
raw_features = {}
raw_labels = {}
fileList = []

for letter in ['A', 'B', 'C', 'E', 'F', 'G', 'H', 'I']: 
    raw_features[f'subj_{letter}'] = np.load(f'pickles/subj_{letter}_features{affix}.npy')
    raw_labels[f'subj_{letter}'] = np.load(f'pickles/subj_{letter}_labels{affix}.npy').reshape(-1)
    fileList.append((f'subj_{letter}', raw_features[f'subj_{letter}'], raw_labels[f'subj_{letter}']))


In [8]:
# separate into "thumb vs all" and "index vs all" ...
for subj, features, labels in fileList:
    thumb_labels = np.where(labels == 1, 1, 0)
    index_labels = np.where(labels == 2, 1, 0)
    middle_labels = np.where(labels == 3, 1, 0)
    ring_labels = np.where(labels == 4, 1, 0)
    pinky_labels = np.where(labels == 5, 1, 0)

[0 3 1 ... 4 2 4]
[0 0 1 ... 0 0 0]
[0 1 3 ... 2 4 4]
[0 1 0 ... 0 0 0]
[4 0 0 ... 2 4 4]
[0 0 0 ... 0 0 0]
[0 3 1 ... 4 2 4]
[0 0 1 ... 0 0 0]
[3 1 4 ... 2 4 4]
[0 1 0 ... 0 0 0]
[0 3 1 ... 2 4 4]
[0 0 1 ... 0 0 0]
[0 3 4 0 4 2 3 3 4 3 3 3 1 0 3 4 2 2 3 1 3 0 4 2 3 2 4 2 0 4 4 4 0 3 1 2 4
 3 1 0 2 4 2 0 0 1 0 1 3 2 1 0 2 0 0 0 4 0 2 0 0 4 2 4 1 2 1 0 2 4 2 1 1 3
 1 0 4 2 1 4 0 3 3 2 3 3 1 3 4 2 2 2 4 3 4 1 3 1 2 4 1 1 1 2 4 2 2 3 1 1 2
 0 4 4 2 1 4 2 2 3 1 0 4 0 1 0 2 3 4 3 0 4 3 4 0 2 2 0 2 4 3 3 4 4 2 0 0 4
 0 2 3 1 0 0 4 1 2 1 3 0 0 4 0 2 0 3 0 2 2 1 1 3 3 2 4 4 2 2 0 2 3 3 0 2 2
 3 2 0 0 0 2 2 3 1 1 3 1 3 1 3 3 4 2 1 2 2 1 0 1 0 0 2 3 3 1 1 3 3 4 2 4 1
 4 2 1 0 2 1 3 4 0 3 1 3 1 2 0 1 3 3 3 2 1 1 4 4 3 4 1 3 1 2 3 4 2 2 0 3 3
 2 0 3 0 0 4 4 2 4 0 3 2 4 2 2 3 1 0 3 4 3 4 0 4 4 2 0 1 2 0 0 0 1 3 2 0 4
 1 2 2 4 0 0 2 4 3 4 4 0 0 4 1 2 0 2 1 0 3 4 4 3 1 2 4 0 0 1 1 3 4 3 2 2 0
 4 1 2 4 0 2 4 1 1 2 3 0 4 0 1 3 0 3 3 3 3 1 3 3 0 2 4 3 0 3 3 3 0 1 3 3 3
 4 1 3 3 1 3 1 1 2 3 1 2 2 0 1 4 3

In [ ]:
#2D CNN w/ Adham's recommendations

import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split

class ConvNet_2D(nn.Module):
    def __init__(self):
        super(ConvNet_2D, self).__init__()
        ### FILL IN ### [10 POINTS]

        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 8, (7, 3)),
            nn.ReLU(), 
            #nn.MaxPool2d(2)
        )
        
        self.conv2 = nn.Sequential(
            nn.Conv2d(8, 16, (5, 3)),
            nn.Conv2d(16, 32, (3, 3)),
            nn.ReLU(),
            #nn.MaxPool2d(2)
        )

        self.conv3 = nn.Sequential(
            nn.ReLU(),
            #nn.MaxPool2d(2)
        )

        self.hidden = nn.Sequential(
            nn.Flatten(),
            # nn.Linear(121728, 256),
            nn.Linear(21056, 256),
            nn.ReLU(),
            nn.Dropout(0.4), # all features norefer
            # nn.Linear(1408, 256), #pseudosampled featrues .
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(128, 5),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        ### FILL IN ### [5 POINTS]
        x = self.conv1(x)
        x = self.conv2(x)
        # x = self.conv3(x)
        x = self.hidden(x)
        return x


In [ ]:
# run CNN
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
# train neural net
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
accuracies = []

for subj, features, labels in fileList:
    features = fft_features[subj]
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size= 0.3) # 70% training 30% test
    conv_net = ConvNet_2D().to(device)
    criterion = nn.CrossEntropyLoss()
    conv_net_optimizer = torch.optim.Adam(conv_net.parameters(), lr=0.001)
    # Train the neural network
    for epoch in range(50):
        # print("epoch = ",epoch)
        conv_net.train()
        data = X_train
        targets = y_train
        data = torch.tensor(data.reshape(int(data.shape[0]), 1, data.shape[1], data.shape[2]), dtype=torch.float32) # reshape # of trial, 1 channel, # of samples
        data = data.to(device)
        targets = torch.tensor(targets, dtype=torch.int64)
        targets = targets.to(device)

        # Forward pass
        conv_net_predictions = conv_net(data)
        conv_net_loss = criterion(conv_net_predictions, targets)

        # Backward pass
        conv_net_optimizer.zero_grad()
        conv_net_loss.backward()
        conv_net_optimizer.step()

    # Evaluate the neural network
    conv_net.eval()
    correct, total = 0, 0
    with torch.no_grad():
        data = X_test
        targets = y_test
        data = torch.tensor(data.reshape(int(data.shape[0]), 1, data.shape[1], data.shape[2]), dtype=torch.float32) # reshape # of trial, 1 channel, # of samples
        data = data.to(device)
        targets = torch.tensor(targets, dtype=torch.int64)
        targets = targets.to(device)
        conv_net_predictions = conv_net(data)
        _, predicted = torch.max(conv_net_predictions.data, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()
    print(f"test accuracy {subj}= ",correct/total)
    accuracies.append(correct/total)

print(f'avg accuracy = {np.mean(accuracies)}')
